**DATA CLEANING**

In [1]:
import pandas as pd
df = pd.read_csv('/content/rapido_july2025_data.csv', parse_dates=['Date']) #change location accordign to your csv location
# Fill missing incomplete-ride reasons
df['Incomplete_Rides_Reason'] = df['Incomplete_Rides_Reason'].fillna('None')
# Drop duplicates and ensure correct types
df.drop_duplicates(inplace=True)
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M').dt.time
df.to_csv('rapido_cleaned.csv', index=False)

/tmp/ipython-input-1129799347.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('/content/rapido_july2025_data.csv', parse_dates=['Date']) #change location accordign to your csv location


**DATA TRANSFORMATION**

In [4]:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv('/content/rapido_july2025_data.csv', parse_dates=['Date']) #change location accordign to your csv location

# Feature Engineering
df['Hour'] = pd.to_datetime(df['Time'], format='%H:%M').dt.hour
df['Weekday'] = pd.to_datetime(df['Date'], dayfirst=True).dt.day_name()
df['Is_Cancelled'] = df['Booking_Status'].isin(['Cancelled', 'Incomplete']).astype(int)

# Customer Segmentation
customer_ride_counts = df.groupby('Customer_ID').size().reset_index(name='Total_Rides')
customer_ride_counts['Customer_Segment'] = pd.cut(
    customer_ride_counts['Total_Rides'],
    bins=[0, 1, 5, df['Customer_ID'].value_counts().max()],
    labels=['One-time', 'Occasional', 'Frequent']
)

# City Flow Matrix
city_flow = df.groupby(['Pickup_Location', 'Drop_Location']).size().unstack(fill_value=0)

# Vehicle Summary
vehicle_summary = df.groupby('Vehicle_Type').agg({
    'Ride_Distance(km)': 'mean',
    'Ride_Time(min)': 'mean',
    'Booking_Value': 'mean'
}).reset_index()

# Save outputs
customer_ride_counts.to_csv('customer_segments.csv', index=False)
city_flow.to_csv('city_matrix.csv')
vehicle_summary.to_csv('vehicle_summary.csv', index=False)


/tmp/ipython-input-171913638.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('/content/rapido_july2025_data.csv', parse_dates=['Date']) #change location accordign to your csv location


**EDA ANALYSIS**

In [8]:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv('/content/rapido_july2025_data.csv', parse_dates=['Date']) #change location accordign to your csv location

# Ride Summary
daily_summary = df.groupby('Date').agg(
    Total_Rides=('Booking_ID', 'count'),
    Completed_Rides=('Booking_Status', lambda x: (x == 'Completed').sum()),
    Cancelled_Rides=('Booking_Status', lambda x: (x == 'Cancelled').sum()),
    Incomplete_Rides=('Booking_Status', lambda x: (x == 'Incomplete').sum()),
    Total_Revenue=('Booking_Value', lambda x: x[df['Booking_Status'] == 'Completed'].sum())
).reset_index()

# Payment Method Summary
payment_summary = df['Payment_Method'].value_counts(normalize=True).mul(100).round(2).reset_index()
payment_summary.columns = ['Payment_Method', 'Percentage']

# Save outputs
daily_summary.to_csv('daily_summary.csv', index=False)
payment_summary.to_csv('payment_summary.csv', index=False)


/tmp/ipython-input-3821251282.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('/content/rapido_july2025_data.csv', parse_dates=['Date']) #change location accordign to your csv location
